In [2]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

z = torch.FloatTensor([1,2,3])

hypothesis = F.softmax(z, dim=0)
print(hypothesis)

hypothesis.sum()

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [8]:
z = torch.rand(3,5, requires_grad=True)

hypothesis = F.softmax(z, dim=1) # 행에 속한 각 열의 합이 1이 되도록
print(hypothesis)
print(hypothesis.sum(dim=1))# 행에 속한 각 열의 합을 구함, 행(dim=0)의 개수가 유지됨


tensor([[0.2025, 0.2176, 0.2324, 0.1142, 0.2333],
        [0.1503, 0.1465, 0.2203, 0.1962, 0.2866],
        [0.1473, 0.1262, 0.2831, 0.1827, 0.2607]], grad_fn=<SoftmaxBackward>)
tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


In [9]:
#이제 각 샘플에 대해서 임의의 레이블을 만듭니다
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 3, 2])


In [15]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)

#Tensor.scatter_(dim, index, src, reduce=None) → Tensor
print(y)
print(y.unsqueeze(1))
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # 맨위의 _ 는 덮어쓰기
print(y_one_hot)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([0, 3, 2])
tensor([[0],
        [3],
        [2]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])


In [18]:
print((y_one_hot * -torch.log(hypothesis)))
print((y_one_hot * -torch.log(hypothesis)).sum(dim=1))
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor([[1.5971, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.6285, 0.0000],
        [0.0000, 0.0000, 1.2618, 0.0000, 0.0000]], grad_fn=<MulBackward0>)
tensor([1.5971, 1.6285, 1.2618], grad_fn=<SumBackward1>)
tensor(1.4958, grad_fn=<MeanBackward0>)


In [30]:
#파이토치로 소프트맥스 비용함수 구현하기
#low level
ls1 = torch.log(F.softmax(z,dim=1))
print(ls1)
ls2 = F.log_softmax(z,dim=1)
print(ls2)

print(y_one_hot)
c1= (y_one_hot * -torch.log(F.softmax(z,dim=1))).sum(dim=1).mean()
c2 = (y_one_hot * -F.log_softmax(z,dim=1)).sum(dim=1).mean()
print(c1)
print(c2)

tensor([[-1.5971, -1.5252, -1.4591, -2.1701, -1.4553],
        [-1.8948, -1.9205, -1.5129, -1.6285, -1.2496],
        [-1.9153, -2.0702, -1.2618, -1.6998, -1.3445]], grad_fn=<LogBackward>)
tensor([[-1.5971, -1.5252, -1.4591, -2.1701, -1.4553],
        [-1.8948, -1.9205, -1.5129, -1.6285, -1.2496],
        [-1.9153, -2.0702, -1.2618, -1.6998, -1.3445]],
       grad_fn=<LogSoftmaxBackward>)
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])
tensor(1.4958, grad_fn=<MeanBackward0>)
tensor(1.4958, grad_fn=<MeanBackward0>)


In [33]:
#여기서 nll이란 Negative Log Likelihood의 약자입니다. 위에서 nll_loss는 F.log_softmax()를 수행한 후에 남은 수식들을 수행
print(y)
c3= F.nll_loss(F.log_softmax(z,dim=1),y)
print(c3)

tensor([0, 3, 2])
tensor(1.4958, grad_fn=<NllLossBackward>)


In [34]:
F.cross_entropy(z, y)



tensor(1.4958, grad_fn=<NllLossBackward>)